# TM10007 Assignment ADNI Dataset
Bram Gerritse 4462599  
Saskia Bijl 4439457  
Karan Ramsodit  
Enzo Kerkhof 44488555

In [17]:
# Run this to use from colab environment to install git repo
!pip install -q --upgrade git+https://github.com/Enzo-Kerkhof/TM10007_PROJECT.git@random_forest

# Use the @ to determine what branch to install. Don't use to install master.

In [0]:
# Import used libraries
import numpy as np

from sklearn import preprocessing
from sklearn import model_selection
from sklearn import decomposition
from sklearn import feature_selection
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [19]:
# Data loading functions. Uncomment the one you want to use
from adni.load_data import load_data

data = load_data()
print('The dataset ABNI contains the following\n')
print(f'The number of samples/patients: {len(data.index)}')
print(f'The number of columns/features: {len(data.columns)}')

The dataset ABNI contains the following

The number of samples/patients: 855
The number of columns/features: 268


## Data preprocessing

In [0]:
# Drop labels from dataframe
X = data.drop(['label'],axis=1)

# Binerize labels AD = 1, CN = 0
Y = preprocessing.label_binarize(data['label'], ['CN', 'AD'])
Y = [i[0] for i in Y]

# Split data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.8)
y_train = np.array(y_train)

In [0]:
# Scale the data to be normal
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [0]:
# Perform a PCA
pca = decomposition.PCA(n_components=100)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [23]:
# Now first use the selectfrom model module. Select all features with a weight above the median.
selector = feature_selection.SelectFromModel(estimator=Lasso(alpha=10**(-10)), threshold='median')
selector.fit(X_train_scaled, y_train)
n_original = X_train_scaled.shape[1]
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)
n_selected = X_train_selected.shape[1]
print(f"Selected {n_selected} from {n_original} features.")


X_train = X_train_selected
X_test = X_test_selected

Selected 134 from 267 features.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.024741115842609532, tolerance: 0.004183625730994154
  positive)


## Hyperparameters

In [37]:
for X, Y in zip(X_train, y_train):
  clf = RandomForestClassifier(n_estimators=50, bootstrap=True, class_weight={0: 1, 1: 1})
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(f"Misclassified: %d / %d" % ((y_test != y_pred).sum(), X_test.shape[0]))

Misclassified: 161 / 684
Misclassified: 157 / 684
Misclassified: 145 / 684
Misclassified: 154 / 684
Misclassified: 154 / 684
Misclassified: 147 / 684
Misclassified: 154 / 684
Misclassified: 161 / 684
Misclassified: 156 / 684
Misclassified: 154 / 684
Misclassified: 162 / 684
Misclassified: 146 / 684
Misclassified: 162 / 684
Misclassified: 157 / 684
Misclassified: 150 / 684
Misclassified: 148 / 684
Misclassified: 157 / 684
Misclassified: 150 / 684
Misclassified: 162 / 684
Misclassified: 152 / 684
Misclassified: 162 / 684
Misclassified: 157 / 684
Misclassified: 166 / 684
Misclassified: 144 / 684
Misclassified: 154 / 684
Misclassified: 167 / 684
Misclassified: 153 / 684
Misclassified: 162 / 684
Misclassified: 153 / 684
Misclassified: 152 / 684
Misclassified: 151 / 684
Misclassified: 159 / 684
Misclassified: 162 / 684
Misclassified: 142 / 684
Misclassified: 158 / 684
Misclassified: 152 / 684
Misclassified: 150 / 684
Misclassified: 164 / 684
Misclassified: 152 / 684
Misclassified: 155 / 684
